# Multiclass Image Classification

## Imports

In [67]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import matplotlib as plt

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
from keras.optimizers import RMSprop
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Rescaling

# model building imports
from keras import Model, Sequential, Input

from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.layers import LeakyReLU
from keras.ops import add
from keras.utils import to_categorical
# model training imports
from keras.optimizers import SGD
from keras.losses import CategoricalCrossentropy
from keras.metrics import CategoricalAccuracy, AUC, F1Score
from keras.callbacks import ModelCheckpoint, CSVLogger, LearningRateScheduler


from keras.layers import RandomContrast, RandomSharpness
from keras.regularizers import l2
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.layers import RandAugment

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image_dataset_from_directory

from library import vis_images

## Metadata Import version 1

In [68]:
# # Caminho base onde estão as pastas com imagens
# base_image_dir = 'C:/Users/luisp/OneDrive/Mestrado/1ano/2sem/Deep Learning (DL)/Project/rare_species 1'

# # Criar coluna com caminho absoluto para cada imagem
# metadata['absolute_path'] = metadata['file_path'].apply(lambda x: os.path.join(base_image_dir, x))

# # Verificar se o ficheiro existe no disco
# metadata['exists_on_disk'] = metadata['absolute_path'].apply(os.path.exists)

# # Mostrar quantos existem e quantos faltam
# total_in_metadata = len(metadata)
# existing_on_disk = metadata['exists_on_disk'].sum()
# real_images_on_disk = sum(len(files) for _, _, files in os.walk(base_image_dir))

# print(f"Total de imagens no metadata: {total_in_metadata}")
# print(f"Total de imagens que existem no disco: {existing_on_disk}")
# print(f"Total real de ficheiros de imagem no disco: {real_images_on_disk}")

# # Mostrar exemplos de imagens em falta
# print("\nExemplos de imagens em falta:")
# print(metadata[~metadata['exists_on_disk']][['file_path']].head())

# # Remover entradas com ficheiros em falta
# metadata_clean = metadata[metadata['exists_on_disk']].drop(columns=['exists_on_disk'])

# # Guardar novo ficheiro limpo
# metadata_clean.to_csv("C:/Users/luisp/OneDrive/Mestrado/1ano/2sem/Deep Learning (DL)/Project/rare_species 1/metadata_clean.csv", index=False)

# print("\n✅ metadata_clean.csv guardado com sucesso — apenas com imagens existentes.")

## Metadata Import 2

Everything is comented because this split was already done in our pcs

#### Inês

In [69]:
metadata_path=r"C:\Users\User\Desktop\deeplearning\metadata.csv"

In [70]:
metadata = pd.read_csv(metadata_path)

In [71]:
metadata.head()

,rare_species_id,eol_content_id,eol_page_id,kingdom,phylum,family,file_path
0,75fd91cb-2881-41cd-88e6-de451e8b60e2,12853737,449393,animalia,mollusca,unionidae,mollusca_unionidae/12853737_449393_eol-full-si...
1,28c508bc-63ff-4e60-9c8f-1934367e1528,20969394,793083,animalia,chordata,geoemydidae,chordata_geoemydidae/20969394_793083_eol-full-...
2,00372441-588c-4af8-9665-29bee20822c0,28895411,319982,animalia,chordata,cryptobranchidae,chordata_cryptobranchidae/28895411_319982_eol-...
3,29cc6040-6af2-49ee-86ec-ab7d89793828,29658536,45510188,animalia,chordata,turdidae,chordata_turdidae/29658536_45510188_eol-full-s...
4,94004bff-3a33-4758-8125-bf72e6e57eab,21252576,7250886,animalia,chordata,indriidae,chordata_indriidae/21252576_7250886_eol-full-s...


In [72]:
metadata.shape

(11983, 7)

In [73]:
# metadata['family'] = metadata['family'].astype(str)  # Ensure it's a string
# metadata['file_path'] = metadata['file_path'].astype(str)  # Ensure path is string

In [74]:
# # Define output directories
# output_base = r"/Users/pedrosantos/Documents 2/Deep Learning/Projeto/rare_species 1/dataset_split"
# os.makedirs(output_base, exist_ok=True)
# for split in ['train', 'val', 'test']:
#     os.makedirs(os.path.join(output_base, split), exist_ok=True)

In [75]:
# # Stratified split at image level
# from sklearn.model_selection import train_test_split

# train_files, temp_files = train_test_split(metadata, test_size=0.3, stratify=metadata['family'], random_state=42)
# val_files, test_files = train_test_split(temp_files, test_size=0.5, stratify=temp_files['family'], random_state=42)

In [76]:
# def move_files(file_subset, split_name):
#     for _, row in file_subset.iterrows():
#         src_path = os.path.join(dataset_directory, row['file_path'])  # Construct full path
#         dst_path = os.path.join(output_base, split_name, row['file_path'])
#         os.makedirs(os.path.dirname(dst_path), exist_ok=True)  # Ensure folder structure
#         if os.path.exists(src_path):
#             shutil.copy2(src_path, dst_path)
#         else:
#             print(f"Warning: {src_path} not found.")

# # Move images to respective folders
# move_files(train_files, 'train')
# move_files(val_files, 'val')
# move_files(test_files, 'test')

# print("Dataset split completed!")

## Localizing Data

#### Pedro

In [77]:
train_path = r"/Users/pedrosantos/Documents 2/Deep Learning/Projeto/rare_species 1/dataset_split/train"
valid_path = r"/Users/pedrosantos/Documents 2/Deep Learning/Projeto/rare_species 1/dataset_split/val"
test_path = r"/Users/pedrosantos/Documents 2/Deep Learning/Projeto/rare_species 1/dataset_split/test"

#### Inês

In [78]:
train_path = r"C:\Users\User\Desktop\deeplearning\dataset_split\train"
valid_path = r"C:\Users\User\Desktop\deeplearning\dataset_split\val"
test_path = r"C:\Users\User\Desktop\deeplearning\dataset_split\test"

## Global Variables

In [79]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

## Data Loading

In [80]:
train = keras.utils.image_dataset_from_directory(
    directory = train_path,
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(224,224),
    color_mode="rgb",
    shuffle=True,
    interpolation = "bilinear"
)

validation = keras.utils.image_dataset_from_directory(
    directory = valid_path,
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(224,224),
    color_mode="rgb",
    shuffle=False,
    interpolation = "bilinear"
)

test = keras.utils.image_dataset_from_directory(
    directory = test_path,
    labels="inferred",
    label_mode="categorical",
    batch_size=32,
    image_size=(224,224),
    color_mode="rgb",
    shuffle=False,
    interpolation = "bilinear"
)

Found 8388 files belonging to 202 classes.
Found 1797 files belonging to 202 classes.
Found 1798 files belonging to 202 classes.


In [81]:
vis_images(train)

NameError: name 'plt' is not defined